Parser

In [357]:
from bs4 import BeautifulSoup
import requests

In [368]:
class parser_recipe:
    def __init__(self, url):
        self.url = url
    
    def parse(self):
        page = requests.get(self.url)

        # get page of the user's recepy
        soup = BeautifulSoup(page.text, "html.parser")

        # find href for printing
        allIngr = soup.find('a', class_='actionsNewPrint')
        url = 'https://povar.ru' + allIngr.attrs['href']
        page = requests.get(url)

        self.soup = BeautifulSoup(new_page.text, "html.parser")
    
    def get_soup(self):
        return self.soup

In [372]:
def recipe(soup):
    recipies = soup.findAll('div', class_ = 'recept_column')

    recipe = []
    for data in recipies:
        if data.find('li') is not None:
            recipe.append(data.text)
    return recipe[0]

In [373]:
# recipe
parser = parser_recipe('https://povar.ru/recipes/obychnyi_plov-46535.html')
parser.parse()
new_soup = parser.get_soup()
recipe = recipe(new_soup)

print(recipe)


Приготовление:


Мясо тщательно промоем и обсушим. Затем нарежем на небольшие кусочки, примерно по 4 см.
Почистим морковку и нарежем тонкими, примерно по 0,5 см в толщину, полосками.
Очистим лук, нарежем тонкими полукольцами или помельче.
Лучше всего плов готовить, конечно же, в казане. Но можно в толстостенной кастрюле. Наливаем в казан или кастрюлю масло и разогреваем на сильном огне. Бросим в разогретое масло немного лука и обжарим его до темно-коричневого цвета. Затем убираем лук из казана.
Выкладываем в разогретое масло мясо.
Минуты через 3 перемешиваем и продолжаем обжаривать на сильном огне до любимой вами степени. Кто-то ведь любит слегка подрумяненное мясо, а кому-то по нраву сильно обжаренные кусочки.
Мясо обжарилось, добавляем лук и перемешиваем. Обжариваем минут 5-7. Не забываем перемешивать, ведь готовим на большом огне!
Теперь добавим полосочки моркови. Все перемешиваем и обжариваем. Готовим либо до подрумянивания моркови, либо до мягкости. Опять же, кому как нравится.
А

In [377]:
def ingred_units(soup):
    # print ingredients and units
    allIngr = soup.findAll('div', class_='ingredients')
    ingrs = []
    ingrs_size = []
    for data in allIngr[0].findAll('p'):
        ingr = ''
        ingr_size = ''
        found = False
        for i in data.text:
            if i == '\n':
                ingr_size = ingr_size + '\n'
                found = False
            if found:
                ingr_size = ingr_size + i
            if not i == '—' and not found:
                ingr = ingr + i
            else:
                found = True
        ingrs.append(ingr.replace(u'\xa0', u''))
        ingrs_size.append(ingr_size.replace(u'\xa0', u''))
    return ingrs, ingrs_size

In [378]:
ingrs, ingrs_size = ingred_units(new_soup)

In [387]:
import re

legal_chars = r'[0-9_,]'
nums = []
for i in ingrs_size:
    n = [re.findall(legal_chars, x) for x in i.split()]
    nums.append(n[0])

In [388]:
def arr_of_tuples_of_ingrs(nums, ingrs_size):
    new_list = []
    for i in num:
        new_num = ''
        if len(i) > 0:
            if len(i) == 1 and i[0] == ',':
                continue
            for j in i:
                new_num += j
        else:
            continue
        new_list.append(new_num)
        
    for i in range(len(new_list)):
        ingrs_size[i] = ingrs_size[i].replace(new_list[i],'')
        new_list[i] = float(new_list[i].replace(',','.'))
    
    return list(zip(ingrs,new_list,ingrs_size))

In [390]:
arr_of_tuples = arr_of_tuples_of_ingrs(nums,ingrs_size)
print(arr_of_tuples)

[('Мякоть свинины', 1.5, 'Килограмм'), ('Рис', 1.0, 'Килограмм(лучше пропаренный)'), ('Лук репчатый', 1.0, 'Килограмм'), ('Морковь', 1.0, 'Килограмм'), ('Масло подсолнечное рафинированное', 300.0, 'Миллилитр'), ('Приправа для плова', 2.0, 'Чайная ложка(по желанию - зира, барбарис, паприка)'), ('Соль', 2.0, 'Чайная ложка(без горки)'), ('Горячая вода', 1.5, 'Литр')]


Damerau-Levenshtein distance

In [332]:
from pyxdameraulevenshtein import normalized_damerau_levenshtein_distance

In [391]:
def find_typos(text, words=['Привет','Яблоко', 'Помидор']):

    rate = 1.0
    final_word = text
    for word in words:
        new_rate = normalized_damerau_levenshtein_distance(text, word)
        if new_rate < rate:
            final_word = word
            rate = new_rate
    if rate > 0.40:
        return text
    if rate == 0.0:
        return
    return f'Возможно, вы имели в виду "{final_word}". Да/нет.'

In [392]:
print(find_typos('Привет'))
print(find_typos('Првет'))
print(find_typos('Прывет'))
print(find_typos('Яблко'))
print(find_typos('Првд'))

None
Возможно, вы имели в виду "Привет". Да/нет.
Возможно, вы имели в виду "Привет". Да/нет.
Возможно, вы имели в виду "Яблоко". Да/нет.
Првд


Recipe standartization.

In [335]:
dict_for_glass = {
    'сахар'   : '200',
    'крахмал' : '160',
    'мука'    : '160',
    'молоко'  : '250',
    'мед'     : '415',
    'масло'   : '225',
    'вода'    : '200',
    'сливки'  : '230',
    'сметана' : '230',
    'рис'     : '180',
    'гречка'  : '165',
    'манка'   : '160'
}

dict_for_tablespoon = {
    'сахар'   : '25',
    'крахмал' : '12',
    'мука'    : '25',
    'молоко'  : '20',
    'мед'     : '30',
    'масло'   : '17',
    'желатин' : '10',
    'сода'    : '8,5',
    'томатная паста' : '24',
    'соль'    : '30'
    
}

dict_for_teaspoon = {
    'сахар'   : '8',
    'крахмал' : '6',
    'мука'    : '8',
    'молоко'  : '5',
    'мед'     : '9',
    'масло'   : '5',
    'соль'    : '10'
}

dict_of_products = {
    'стакан' : dict_for_glass,
    'столовая ложка' : dict_for_tablespoon,
    'чайная ложка' : dict_for_teaspoon
}

In [398]:
user_ingr = 'Мука'
user_quantity = 2
user_measure = 'стакан'
user_list = [(user_ingr,user_quantity,user_measure)]

In [416]:
def normalization(user_list):
    found = False
    for i in range(len(user_list)):
        new_measure = 0
        ingrt    = user_list[i][0].lower()
        quantity = user_list[i][1]
        measure  = user_list[i][2].lower()

        if dict_of_products.get(measure):
            if measure == 'чайная ложка':
                meas = dict_of_products['чайная ложка'][ingrt]
                new_measure = float(meas)*quantity

            if measure == 'столовая ложка':
                meas = dict_of_products['столовая ложка'][ingrt]
                new_measure = float(meas)*quantity

            if measure == 'стакан':
                meas = dict_of_products['стакан'][ingrt]
                new_measure = float(meas)*quantity
            else:
                continue
            user_list[i] = (ingrt, new_measure,'граммы')

In [417]:
normalization(user_list)

In [418]:
user_list

[('мука', 320.0, 'граммы')]

In [47]:
from icrawler.builtin import GoogleImageCrawler

google_crawler = GoogleImageCrawler(storage={'root_dir': 'D:\\nir'})
google_crawler.crawl(keyword='Памятник Екатерине 2 спб', max_num=200)

INFO:icrawler.crawler:start crawling...
INFO:icrawler.crawler:starting 1 feeder threads...
INFO:feeder:thread feeder-001 exit
INFO:icrawler.crawler:starting 1 parser threads...
INFO:icrawler.crawler:starting 1 downloader threads...
INFO:parser:parsing result page https://www.google.com/search?q=%D0%9F%D0%B0%D0%BC%D1%8F%D1%82%D0%BD%D0%B8%D0%BA+%D0%95%D0%BA%D0%B0%D1%82%D0%B5%D1%80%D0%B8%D0%BD%D0%B5+2+%D1%81%D0%BF%D0%B1&ijn=0&start=0&tbs=&tbm=isch
INFO:downloader:image #1	https://upload.wikimedia.org/wikipedia/commons/c/c1/Monument_to_Catherine_II_in_SPB.jpg
INFO:downloader:image #2	https://fs.tonkosti.ru/48/qe/48qe4qfvlgmcwc4s8wco8g4ok.jpg
INFO:downloader:image #3	http://visit-petersburg.ru/media/uploads/tourobject/196977/196977_cover.jpg
INFO:downloader:image #4	http://www.peterburg.biz/images/dost/ek2b.jpg
INFO:downloader:image #5	https://mos-holidays.ru/spb/wp-content/uploads/sites/2/2018/09/pamyatnik-ekaterine-ii_1.jpg
INFO:downloader:image #6	https://www.votpusk.ru/gallery/mobile/32

INFO:downloader:image #60	http://www.zdravrussia.ru/images/nnews/huge/6558.jpg
INFO:downloader:image #61	https://www.bankgorodov.ru/public//photos/sights/1422888603.jpg
INFO:downloader:image #62	https://autotravel.ru/phalbum/91075/132.jpg
INFO:downloader:image #63	https://d1ualep0003st.cloudfront.net/uploads/cards/images/xlarge/ac/10/067116b2-5b21-11e7-b8d2-000000000000.jpg
INFO:downloader:image #64	https://wikiway.com/upload/resize_cache/hl-photo/a87/41a/1024_800_1/pamyatnik_ekaterine_ii_21.jpg
INFO:downloader:image #65	https://upload.gradpetra.net/photo/photo_spb/pamyatnik/1/web/68737.jpg
INFO:downloader:image #66	https://rus-towns.ru/wp-content/gallery/sankt-peterburg-pamyatnik-ekaterine-velikoj-20-07-2011/spb-pamyatnik-ekaterine-ii-02.jpg
INFO:downloader:image #67	https://cf.ppt-online.org/files1/slide/a/av4IqFeUkfxAuiY2gbOVyhpNm98ocSzQlGWCrHK3Ds/slide-0.jpg
INFO:downloader:image #68	http://worldwalk.info/preview_img/4640-mid.jpg
INFO:downloader:image #69	https://dic.academic.ru/pi

In [54]:
import csv

with open("D:\\nir\\admiralteistvo") as f:
    print("")

PermissionError: [Errno 13] Permission denied: 'D:\\nir\\admiralteistvo'

In [16]:
# Instantiate the scraper objects 

google_hashtag = Hashtag('palacesquare')

# Scrape their respective data 


google_hashtag.scrape()


print(google_hashtag.download('D:/nir'))

AttributeError: 'Hashtag' object has no attribute 'download'

In [17]:
profile_pic_url = google_hashtag.profile_pic_url

In [18]:
profile_pic_url

'https://scontent-hel3-1.cdninstagram.com/v/t51.2885-15/e35/c0.100.1440.1440a/s150x150/183166196_508703303830444_4718350219440811693_n.jpg?_nc_ht=scontent-hel3-1.cdninstagram.com&_nc_cat=104&_nc_ohc=49Mt-_qDTm4AX9mPzsO&edm=ABZsPhsBAAAA&ccb=7-4&oh=787ae877445b93405b7ffe403afc2a05&oe=61A24A14&_nc_sid=4efc9f'

In [19]:
for i in profile_pic_url:
    print(i)

h
t
t
p
s
:
/
/
s
c
o
n
t
e
n
t
-
h
e
l
3
-
1
.
c
d
n
i
n
s
t
a
g
r
a
m
.
c
o
m
/
v
/
t
5
1
.
2
8
8
5
-
1
5
/
e
3
5
/
c
0
.
1
0
0
.
1
4
4
0
.
1
4
4
0
a
/
s
1
5
0
x
1
5
0
/
1
8
3
1
6
6
1
9
6
_
5
0
8
7
0
3
3
0
3
8
3
0
4
4
4
_
4
7
1
8
3
5
0
2
1
9
4
4
0
8
1
1
6
9
3
_
n
.
j
p
g
?
_
n
c
_
h
t
=
s
c
o
n
t
e
n
t
-
h
e
l
3
-
1
.
c
d
n
i
n
s
t
a
g
r
a
m
.
c
o
m
&
_
n
c
_
c
a
t
=
1
0
4
&
_
n
c
_
o
h
c
=
4
9
M
t
-
_
q
D
T
m
4
A
X
9
m
P
z
s
O
&
e
d
m
=
A
B
Z
s
P
h
s
B
A
A
A
A
&
c
c
b
=
7
-
4
&
o
h
=
7
8
7
a
e
8
7
7
4
4
5
b
9
3
4
0
5
b
7
f
f
e
4
0
3
a
f
c
2
a
0
5
&
o
e
=
6
1
A
2
4
A
1
4
&
_
n
c
_
s
i
d
=
4
e
f
c
9
f
